In [17]:
import numpy as np
import json
import re
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F

import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)

In [19]:
def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())

In [20]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [21]:
class ChatDataset(Dataset):
    def __init__(self, intents, all_words, tags):
        self.intents = intents
        self.all_words = all_words
        self.tags = tags
        self.xy = self._prepare_data()

    def _prepare_data(self):
        xy = []
        for intent in self.intents:
            tag = intent['tag']
            for pattern in intent['patterns']:
                tokenized_sentence = tokenize(pattern)
                bag = bag_of_words(tokenized_sentence, self.all_words)
                label = self.tags.index(tag)
                xy.append((bag, label))
        return xy

    def __getitem__(self, index):
        bag, label = self.xy[index]
        return bag, label

    def __len__(self):
        return len(self.xy)

In [22]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Dropout avec taux de 0.5

        # Initialisation des poids avec la méthode de He
        torch.nn.init.kaiming_uniform_(self.l1.weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.l2.weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.l3.weight)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.dropout(out)  # Dropout après la première couche cachée
        out = self.l2(out)
        out = self.relu(out)
        out = self.dropout(out)  # Dropout après la deuxième couche cachée
        out = self.l3(out)
        return out

In [23]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        tokenized_sentence = tokenize(pattern)
        all_words.extend(tokenized_sentence)
        xy.append((tokenized_sentence, tag))

ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

batch_size = 16
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 512
output_size = len(tags)

dataset = ChatDataset(intents['intents'], all_words, tags)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


num_epochs = 2500
correct=0
total=0
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        correct += (outputs.argmax(dim=1) == labels).sum().item()

    if (epoch+1) % 100 == 0:
        accuracy =  100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy}')

Epoch [100/2500], Loss: 0.0149, Accuracy: 92.66666666666667
Epoch [200/2500], Loss: 0.0004, Accuracy: 95.91056910569105
Epoch [300/2500], Loss: 0.0004, Accuracy: 97.00542005420054
Epoch [400/2500], Loss: 0.0003, Accuracy: 97.53658536585365
Epoch [500/2500], Loss: 0.0001, Accuracy: 97.84065040650407
Epoch [600/2500], Loss: 0.0000, Accuracy: 98.05420054200542
Epoch [700/2500], Loss: 0.0000, Accuracy: 98.20557491289199
Epoch [800/2500], Loss: 0.0000, Accuracy: 98.33231707317073
Epoch [900/2500], Loss: 0.0000, Accuracy: 98.42276422764228
Epoch [1000/2500], Loss: 0.0003, Accuracy: 98.50162601626016
Epoch [1100/2500], Loss: 0.0000, Accuracy: 98.56393200295639
Epoch [1200/2500], Loss: 0.0000, Accuracy: 98.60907859078591
Epoch [1300/2500], Loss: 0.0921, Accuracy: 98.6541588492808
Epoch [1400/2500], Loss: 0.0000, Accuracy: 98.69221835075494
Epoch [1500/2500], Loss: 0.0000, Accuracy: 98.72195121951219
Epoch [1600/2500], Loss: 0.0000, Accuracy: 98.7489837398374
Epoch [1700/2500], Loss: 0.0000, Ac

In [24]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
    }

FILE = "chatbot_model_base.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to chatbot_model_base.pth


## Deploying Q/A model

In [25]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# Charger le dataset JSON
with open('piaf-v1.1.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# Prétraitement des données
paragraphs = []
questions = []
answers_start = []
answers_end = []

for article in dataset['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            for answer in qa['answers']:
                answer_start = answer['answer_start']
                answer_end = answer_start + len(answer['text'])

                paragraphs.append(context)
                questions.append(question)
                answers_start.append(answer_start)
                answers_end.append(answer_end)

In [27]:
# Chargement du modèle de question-réponse
model_qa = AutoModelForQuestionAnswering.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
tokenizer_qa = AutoTokenizer.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")

In [28]:
# Fonction pour générer une réponse à partir d'une question
def generate_answer(question, context):
    # Prétraitement des données
    inputs = tokenizer_qa(question, context, add_special_tokens=True, return_tensors="pt")
    
    # Prédiction de la réponse
    outputs = model_qa(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    
    # Obtenir les indices de début et de fin de la réponse prédite
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    
    # Convertir les indices en texte
    answer = tokenizer_qa.convert_tokens_to_string(tokenizer_qa.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
    
    return answer

In [29]:
# Fonction pour trouver le contexte le plus pertinent pour une question donnée
def find_most_relevant_context(question, dataset):
    # Utiliser TF-IDF pour représenter les contextes en vecteurs
    contexts=[]
    for article in dataset['data']:
        for paragraph in article['paragraphs']:
            contexts += [paragraph['context']]
    
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(contexts)
    
    # Convertir la question en vecteur
    question_vector = vectorizer.transform([question])
    
    # Calculer les similarités entre la question et les contextes
    similarities = cosine_similarity(question_vector, vectors)
    
    # Trouver l'index du contexte le plus similaire
    most_similar_index = similarities.argmax()
    
    # Renvoyer le contexte le plus similaire
    return contexts[most_similar_index]

## Test

In [16]:
import random
import json
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Charger les données et le modèle pré-entraîné
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "chatbot_model_base.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# Charger le modèle
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# Nom du chatbot
bot_name = "QueryBot"

print("Début de la conversation ('quit' pour finir la conversation...)")
while True:
    # Saisie utilisateur
    sentence = input("User: ")
    if sentence == 'quit':
        break
    
    # Prédiction de l'intention
    sentence_qa = sentence
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    
    # Répondre en fonction de l'intention prédite ou en utilisant la question-réponse
    if prob.item() > 0.75:  # ajuster le seuil de probabilité selon la précision souhaitée
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
                break  # sortir après avoir trouvé une réponse appropriée
    else:
        response = process_question(sentence_qa)
        if response:
            print(f"{bot_name}: {response}")
        else:
            context = find_most_relevant_context(sentence_qa, dataset)
            answer = generate_answer(sentence_qa, context)
            print(f"{bot_name}: {answer}")


Début de la conversation ('quit' pour finir la conversation...)
User: bonjour
QueryBot: Content de vous revoir
User: ça va?
QueryBot: Tout va bien.. Et vous?
User: bien
QueryBot: Content de le savoir!
User: blague
QueryBot: Un perfectionniste est entre dans un bar... apparemment, le bar n'etait pas assez haut
User: devinett
QueryBot: Que trouverez-vous reellement a la fin de chaque arc-en-ciel ?.....La lettre 'w'


KeyboardInterrupt: Interrupted by user

In [30]:
# Test
import random
import json
import torch
from pymongo import MongoClient
from datetime import datetime
import re
from collections import defaultdict
from dateutil.parser import parse

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Charger les données et le modèle pré-entraîné (à partir de intents.json et du modèle de chatbot)
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "chatbot_model_base.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# Charger le modèle
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# Nom du chatbot
bot_name = "QueryBot"

# Fonction de prétraitement du texte
def preprocess_text(text):
    cleaned_text = text.lower().strip()
    return cleaned_text

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['StageTT']
collection = db['TT']

# Charger les articles depuis MongoDB et les prétraiter
articles = list(collection.find())

for article in articles:
    article['Content'] = preprocess_text(article['Content'])
    if isinstance(article['Publish Date'], str):
        try:
            article['Publish Date'] = parse(article['Publish Date']).replace(tzinfo=None)
        except ValueError as e:
            print(f"Error parsing date: {e}, article: {article}")
            article['Publish Date'] = None

# Filtrer les articles avec des dates invalides
articles = [article for article in articles if article['Publish Date'] is not None]

# Categoriser les articles par année et sentiment
articles_by_year = defaultdict(lambda: {"positive": 0, "negative": 0})

for article in articles:
    year = article['Publish Date'].year
    sentiment = article.get('SentimentCamembert', 'positive').lower()
    if sentiment == 'positive':
        articles_by_year[year]['positive'] += 1
    elif sentiment == 'negative':
        articles_by_year[year]['negative'] += 1

print("Début de la conversation ('quit' pour finir la conversation...)")
while True:
    user_input = input("User: ").strip().lower()
    if user_input == 'quit':
        break

    # Prédiction de l'intention à l'aide du modèle de chatbot
    sentence_qa = user_input
    sentence = tokenize(user_input)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    # Répondre en fonction de l'intention prédite
    if prob.item() > 0.75:  # ajuster le seuil de probabilité selon la précision souhaitée
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
                break  # sortir après avoir trouvé une réponse appropriée
    else:
        # Répondre en se basant sur les données de la collection MongoDB TT
        if "tunisie télécom" in user_input:
            if articles:
                latest_article = max(articles, key=lambda x: x['Publish Date'])
                print(f"Chatbot: La dernière nouvelle sur Tunisie Télécom est : {latest_article['Title']}")
            else:
                print("Chatbot: Il n'y a pas d'articles disponibles sur Tunisie Télécom.")
        elif "combien d'articles" in user_input and "positifs" in user_input:
            year_match = re.search(r'\b\d{4}\b', user_input)
            if year_match:
                year = int(year_match.group(0))
                count = articles_by_year[year]['positive']
                print(f"Chatbot: Il y a {count} articles positifs en {year}.")
            else:
                print("Chatbot: Veuillez spécifier une année.")
        elif "combien d'articles" in user_input and "négatifs" in user_input:
            year_match = re.search(r'\b\d{4}\b', user_input)
            if year_match:
                year = int(year_match.group(0))
                count = articles_by_year[year]['negative']
                print(f"Chatbot: Il y a {count} articles négatifs en {year}.")
            else:
                print("Chatbot: Veuillez spécifier une année.")
        elif "année avec le plus d'articles négatifs" in user_input:
            year = max(articles_by_year, key=lambda x: articles_by_year[x]['negative'])
            count = articles_by_year[year]['negative']
            print(f"Chatbot: L'année avec le plus d'articles négatifs est {year} avec {count} articles.")
        elif "année avec le plus d'articles positifs" in user_input:
            year = max(articles_by_year, key=lambda x: articles_by_year[x]['positive'])
            count = articles_by_year[year]['positive']
            print(f"Chatbot: L'année avec le plus d'articles positifs est {year} avec {count} articles.")
        else:
            # Aucune intention prédite, ni réponse directe trouvée
            print("Chatbot: Désolé, je ne comprends pas votre question.")



Début de la conversation ('quit' pour finir la conversation...)
User: Quelle est la dernière nouvelle sur Tunisie Télécom ?
QueryBot: ...
User: Quelle est la dernière nouvelle sur Tunisie Télécom 
QueryBot: ...


KeyboardInterrupt: Interrupted by user